In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-13-04.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-13-42.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-12-54.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-12-25.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-14-39.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-13-36.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 02-11-55.png
/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-12-12.png


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers

In [4]:
from transformers import pipeline

2024-07-30 17:01:11.118868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 17:01:11.118979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 17:01:11.254068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
cv_image = '/kaggle/input/images-to-use/Screenshot from 2024-07-30 02-11-55.png'
hack_image = '/kaggle/input/images-to-use/Screenshot from 2024-07-30 15-12-25.png'

In [38]:
nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

nlp(
    hack_image,
    "summarize the document"
)

[{'score': 0.966942310333252,
  'answer': 'HACKATHON TIMELINES',
  'start': 69,
  'end': 70}]

# Fine Tune for Financial Invoices

In [5]:
model_out = "impira/layoutlm-document-qa"
batch_size = 4

In [6]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceM4/DocumentVQA")
ds

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

...

Generating train split:   0%|          | 0/39463 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5349 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5188 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/51 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 39463
    })
    validation: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 5349
    })
    test: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 5188
    })
})

In [7]:
ds["train"].features

{'questionId': Value(dtype='int32', id=None),
 'question': Value(dtype='string', id=None),
 'question_types': [Value(dtype='string', id=None)],
 'image': Image(mode=None, decode=True, id=None),
 'docId': Value(dtype='int32', id=None),
 'ucsf_document_id': Value(dtype='string', id=None),
 'ucsf_document_page_no': Value(dtype='string', id=None),
 'answers': [Value(dtype='string', id=None)]}

In [8]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_out)

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
from transformers import AutoModelForDocumentQuestionAnswering

model = AutoModelForDocumentQuestionAnswering.from_pretrained(model_out)

model.safetensors:   0%|          | 0.00/511M [00:00<?, ?B/s]

In [11]:
from transformers import TrainingArguments

# REPLACE THIS WITH YOUR REPO ID
repo_id = "pacman2223/univ-docu-model-v2"

training_args = TrainingArguments(
    output_dir=repo_id,
    per_device_train_batch_size=4,
    num_train_epochs=20,
    save_steps=100,
    logging_steps=50,
    eval_strategy="steps",
    learning_rate=5e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
)

In [12]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=processor,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: too many dimensions 'str'